In [ ]:
!gsutil cp -r gs://ddsp/models/ .
!rclone copy models/ remote:/content/drive/MyDrive/MLProyect/ModelosEntretandos


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!gsutil cp -r gs://ddsp/models/ .


### Instructions for running:

* Make sure to use a GPU runtime, click:  __Runtime >> Change Runtime Type >> GPU__
* Press ▶️ on the left of each of the cells
* View the code: Double-click any of the cells
* Hide the code: Double click the right side of the cell





In [ ]:
!sudo apt-get install portaudio19-dev
!pip install sounddevice
!apt install -y ffmpeg
!pip install --upgrade librosa

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
portaudio19-dev is already the newest version (19.6.0-1.1).
0 upgraded, 0 newly installed, 0 to remove and 23 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 23 not upgraded.


In [ ]:
#@title #Install and Import

#@markdown Install ddsp, define some helper functions, and download the model. This transfers a lot of data and _should take a minute or two_.


import copy
import os
import time
import sounddevice as sd
import scipy.io.wavfile as wav
import numpy as np
import matplotlib.pyplot as plt
import librosa
import librosa.display
from IPython.display import Audio
import gin
from google.colab import files
import librosa
import matplotlib.pyplot as plt
import numpy as np
import pickle
import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds


In [ ]:
#@title Record or Upload Audio with Pitch Detection
#@markdown * Either record audio from microphone or upload audio from file (.mp3 or .wav)
#@markdown * Audio should be monophonic (single instrument / voice)
import sounddevice as sd
import scipy.io.wavfile as wav
import numpy as np
import matplotlib.pyplot as plt
import librosa
import librosa.display


def record_audio(duration, fs=44100):
    recording = sd.rec(int(duration * fs), samplerate=fs, channels=1)
    sd.wait()
    return recording[:, 0]

def load_audio(filename):
    return librosa.load(filename, sr=None, mono=True)[0]

def extract_features(audio, sr):
    f0 = librosa.yin(audio, fmin=librosa.note_to_hz('C2'), fmax=librosa.note_to_hz('C7'))
    loudness = librosa.feature.rms(y=audio)
    return f0, loudness

def plot_features(f0, loudness):
    frames = range(len(f0))
    t = librosa.frames_to_time(frames)
    plt.figure()
    plt.subplot(2, 1, 1)
    plt.plot(t, f0, label='f0')
    plt.xlabel('Time (s)')
    plt.ylabel('Frequency (Hz)')
    plt.title('f0')
    plt.subplot(2, 1, 2)
    plt.plot(t, loudness[0], label='loudness')
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude')
    plt.title('Loudness')
    plt.tight_layout()
    plt.show()

# Grabar o cargar audio
option = input("Escribe 'R' para grabar audio o 'C' para cargar un archivo de audio: ")
if option.upper() == 'R':
    duration = int(input("Ingresa la duración de la grabación en segundos: "))
    audio = record_audio(duration)
    sr = 44100
elif option.upper() == 'C':
    filename = input("Ingresa el nombre del archivo de audio: ")
    audio = load_audio(filename)
    sr = librosa.get_samplerate(filename)
else:
    print("Opción no válida.")


# Extraer y trazar características
if audio is not None:
    f0, loudness = extract_features(audio, sr)
    plot_features(f0, loudness)

# Calcula las magnitudes del espectrograma
D = np.abs(librosa.stft(audio))
magnitudes = librosa.amplitude_to_db(D, ref=np.max)

# Convierte el audio a una matriz de NumPy de tipo flotante
audio_np_float = np.array(audio, dtype=float)

# Grafica el espectrograma con detección de tono
plt.figure(figsize=(12, 8))
librosa.display.specshow(magnitudes, y_axis='log', x_axis='time')
plt.colorbar(format='%+2.0f dB')
plt.title('Spectrogram with Pitch Detection')
plt.show()

# Grafica la forma de onda del audio
plt.figure(figsize=(12, 4))
librosa.display.waveshow(audio_np_float, sr=sr)
plt.title('Waveform')
plt.show()

from IPython.display import Audio
# Reproducir el audio
Audio(audio, rate=sr)


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
#@title Choose an instrument (load a model)
#@markdown Run for every new audio input
model = 'Tenor_Saxophone' #@param ['Violin', 'Flute', 'Flute2', 'Trumpet', 'Tenor_Saxophone', 'Upload your own (checkpoint folder as .zip)']
MODEL = model

def find_model_dir(dir_name):
  # Iterate through directories until model directory is found
  for root, dirs, filenames in os.walk(dir_name):
    for filename in filenames:
      if filename.endswith(".gin") and not filename.startswith("."):
        model_dir = root
        break
  return model_dir

if model in ('Violin', 'Flute', 'Flute2', 'Trumpet', 'Tenor_Saxophone'):
  # Pretrained models.
  PRETRAINED_DIR = '/content/pretrained'
  # Copy over from gs:// for faster loading.
  !rm -r $PRETRAINED_DIR &> /dev/null
  !mkdir $PRETRAINED_DIR &> /dev/null
  GCS_CKPT_DIR = 'gs://ddsp/models/timbre_transfer_colab/2021-01-06'
  model_dir = os.path.join(GCS_CKPT_DIR, 'solo_%s_ckpt' % model.lower())

  !gsutil cp $model_dir/* $PRETRAINED_DIR &> /dev/null
  model_dir = PRETRAINED_DIR
  gin_file = os.path.join(model_dir, 'operative_config-0.gin')

else:
  # User models.
  UPLOAD_DIR = '/content/uploaded'
  !mkdir $UPLOAD_DIR
  uploaded_files = files.upload()

  for fnames in uploaded_files.keys():
    print("Unzipping... {}".format(fnames))
    !unzip -o "/content/$fnames" -d $UPLOAD_DIR &> /dev/null
  model_dir = find_model_dir(UPLOAD_DIR)
  gin_file = os.path.join(model_dir, 'operative_config-0.gin')


# Load the dataset statistics.
DATASET_STATS = None
dataset_stats_file = os.path.join(model_dir, 'dataset_statistics.pkl')
print(f'Loading dataset statistics from {dataset_stats_file}')
try:
  if tf.io.gfile.exists(dataset_stats_file):
    with tf.io.gfile.GFile(dataset_stats_file, 'rb') as f:
      DATASET_STATS = pickle.load(f)
except Exception as err:
  print('Loading dataset statistics from pickle failed: {}.'.format(err))


# Parse gin config,
with gin.unlock_config():
  gin.parse_config_file(gin_file, skip_unknown=True)

# Assumes only one checkpoint in the folder, 'ckpt-[iter]`.
ckpt_files = [f for f in tf.io.gfile.listdir(model_dir) if 'ckpt' in f]
ckpt_name = ckpt_files[0].split('.')[0]
ckpt = os.path.join(model_dir, ckpt_name)

# Ensure dimensions and sampling rates are equal
time_steps_train = gin.query_parameter('F0LoudnessPreprocessor.time_steps')
n_samples_train = gin.query_parameter('Harmonic.n_samples')
hop_size = int(n_samples_train / time_steps_train)

time_steps = int(audio.shape[1] / hop_size)
n_samples = time_steps * hop_size

# print("===Trained model===")
# print("Time Steps", time_steps_train)
# print("Samples", n_samples_train)
# print("Hop Size", hop_size)
# print("\n===Resynthesis===")
# print("Time Steps", time_steps)
# print("Samples", n_samples)
# print('')

gin_params = [
    'Harmonic.n_samples = {}'.format(n_samples),
    'FilteredNoise.n_samples = {}'.format(n_samples),
    'F0LoudnessPreprocessor.time_steps = {}'.format(time_steps),
    'oscillator_bank.use_angular_cumsum = True',  # Avoids cumsum accumulation errors.
]

with gin.unlock_config():
  gin.parse_config(gin_params)


# Trim all input vectors to correct lengths
for key in ['f0_hz', 'f0_confidence', 'loudness_db']:
  audio_features[key] = audio_features[key][:time_steps]
audio_features['audio'] = audio_features['audio'][:, :n_samples]


# Set up the model just to predict audio given new conditioning
model = ddsp.training.models.Autoencoder()
model.restore(ckpt)

# Build model by running a batch through it.
start_time = time.time()
_ = model(audio_features, training=False)
print('Restoring model took %.1f seconds' % (time.time() - start_time))

In [ ]:
#@title #Resynthesize Audio
# Calcula la STFT del audio
D = librosa.stft(audio)

# Calcula la magnitud y la fase
magnitude, phase = librosa.magphase(D)

# Estima f0 utilizando el modelo CREPE
f0 = librosa.yin(audio, fmin=librosa.note_to_hz('C2'), fmax=librosa.note_to_hz('C7'))

# Modifica la magnitud
bins_per_octave = 100  #@param {type:"number", min:1, max:100, step:1}
cqt = librosa.cqt(audio, sr=sr, bins_per_octave=bins_per_octave)
fmin = librosa.note_to_hz('C1')
bins = np.round(bins_per_octave * np.log2(f0 / fmin))
magnitude *= np.greater_equal.outer(bins, np.arange(magnitude.shape[0])).T

# Reconstruye la STFT
D_modified = magnitude * phase

# Realiza la inversa de la STFT para obtener el audio resintetizado
audio_resynthesized = librosa.istft(D_modified)

# Reproduce el audio
Audio(audio_resynthesized, rate=sr)

Output hidden; open in https://colab.research.google.com to view.